In [1]:
## import all required libraries
import pandas as pd # for data manipulation 
import numpy as np # for numerical calculation
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for advance data visualization

from pulp import* # for solve the linear programing  problem
from sqlalchemy import create_engine # to connect the sql database 

In [2]:
# Set the maximum number of columns to be displayed
pd.set_option('display.max_columns', None)

In [3]:
# Load the data 
sand = pd.read_excel(r'C:\Users\Dell\Desktop\DS Project\Robo Sand(New Data).xlsx')

In [4]:
sand.rename(columns = {'Customer/Vendor Name':"Party Name",'Distribution Rule':"Warehouse",'U Km':"Distance","Quantity":"Net Weight"},inplace = True)

In [5]:
# check the null values and their sum
sand.isna().sum()
sand = sand.dropna() # drop null values

In [6]:
# unique warehouses and party_names
warehouses = sand['Warehouse'].unique()
party_names = sand['Party Name'].unique()

In [7]:
# shows Unique Warehouses and Party Names
print(warehouses)
print("\n")
print(party_names)

['GIRMAPUR' 'GIRMAPR2' 'KEESARA3' 'LKDRM2' 'LKDRM4' 'RSDSH' 'SLKPY']


['URBANRISE LIFESTYLES PRIVATE LIMITED - IDA BOLLARAM'
 'MUPPA PROJECTS INDIA PVT LTD - OSMAN NAGAR'
 'SHAPOORJI PALLONJI AND COMPANY PRIVATE LIMITED - KANDI' ...
 'SATYAVARDHAN REDDY VALLAM REDDY - KONDAPUR'
 'SRIJA CONSTRUCTIONS - NALLAGANDLA'
 'VISHEESHTA INFRACON LLP - MOUTHNAGAR']


In [8]:
# Create the cost matric with respective to Warehouse and party cosider minimum cost per KM per MT in ₹
cost_mat = sand.pivot_table(values = 'Freight Rate', index = 'Warehouse', columns = 'Party Name', aggfunc = 'min')

In [9]:
# fill with high value (100000) wherever not supplied the sand  
cost_mat.fillna(100000, inplace = True)
cost_mat.head()

In [10]:
# Create a pivot table with Warehouse as index, Party Name as columns, and Distance as values
distance_matrix = sand.pivot_table(values='Distance', index='Warehouse', columns='Party Name')
# fill with 1000 Wherever no supply
distance_matrix.fillna(1000, inplace = True)
distance_matrix.head()

In [11]:
# create pivot_table for quantity of sand transporte index as Warehouse and columns Party values are Net_weight
weight_mat = sand.pivot_table(values = 'Net Weight', index = 'Warehouse', columns = 'Party Name', aggfunc =sum )
# fill with zeros wherever no sulpply 
weight_mat = weight_mat.fillna(0)
weight_mat.head()

In [12]:
# create pivot table with warehouse as index and net weight sum
supply = pd.pivot_table(sand, values='Net Weight', index=['Warehouse'], aggfunc=sum, margins=True)
supply

In [13]:
#create a pivot_tabel for total demand of each party (Before optimization the Quantity of sand Transport from Warehouse the Party)
demand = pd.pivot_table(sand, values='Net Weight', index ='Warehouse', columns ='Party Name', aggfunc = sum, margins =True, margins_name='Grand Total')
demand

In [ ]:
# Only consider the Demand 
demand = demand.iloc[7:]
# Rename the column
demand.rename(index= {'Grand Total': 'Demand'}, inplace = True)
demand.head()

In [ ]:
# Define the problem
prob = LpProblem("Transportation Problem", LpMinimize)

In [ ]:
# Define decision variables
route_vars = LpVariable.dicts("Route", (warehouses, party_names), lowBound=0, cat='Continuous')
route_vars

In [ ]:
# Define the objective function
prob += lpSum([route_vars[w][p] * cost_mat.loc[w][p] * distance_matrix.loc[w][p] for w in warehouses for p in party_names]), " Transportation Cost"

In [ ]:
# Define the constraints supply(Actual supplied == Warehouse_Capacity )
for w in warehouses:
    prob += lpSum([route_vars[w][p]   for p in party_names]) <= supply.loc[w] , "Sum of quatity supplied from Warehouse to paty %s" % w

In [ ]:
demand2 = demand.T
print(demand2.head(10))

In [ ]:
# demand constraints(The Quantity of sand Recivied = Actual Demand)
for p in party_names:
    prob+= lpSum([route_vars[w][p] for w in warehouses]) >= demand2.loc[p],'sum of Demand of the party %s'%p

In [21]:
# Execute the problem
prob.solve()
print('Total_transportation_Costs = {:,} '.format(int(value(prob.objective))))

Total_transportation_Costs = 277,663,817 


In [22]:
# shows the status of the Problem
print(f"Status: {LpStatus[prob.status]}")

Status: Optimal


In [23]:
# Create DataFrame 
decision_var_df = pd.DataFrame(index=distance_matrix.index, columns=distance_matrix.columns, dtype='float')

In [24]:
# Fill the DataFrame with the optimized values of the decision variables
for w in distance_matrix.index:
    for p in distance_matrix.columns:
        decision_var_df.loc[w, p] = route_vars[w][p].varValue

In [25]:
# Decision_Variables(How Much Quantity of Sand Transport form Warehouse to Party )
decision_var_df.head()

Party Name  1000 PILLAR S CONSTRUCTIONS - HAFEEZPET.  \
Warehouse                                              
GIRMAPR2                                        0.00   
GIRMAPUR                                      495.36   
KEESARA3                                        0.00   
LKDRM2                                          0.00   
LKDRM4                                          0.00   

Party Name  1000 PILLARS ORCHID DEVELOPERS  \
Warehouse                                    
GIRMAPR2                              0.00   
GIRMAPUR                              0.00   
KEESARA3                             18.82   
LKDRM2                                0.00   
LKDRM4                                0.00   

Party Name  2D PHARMACEUTICALS PRIVATE LIMITED -  MALLAPUR  \
Warehouse                                                    
GIRMAPR2                                              0.00   
GIRMAPUR                                              0.00   
KEESARA3                                            261.28   
LKDRM2                                                0.00   
LKDRM4                                                0.00   

Party Name  2D PHARMACEUTICALS PRIVATE LIMITED - BANDLAGUDA  \
Warehouse                                                     
GIRMAPR2                                               0.00   
GIRMAPUR                                               0.00   
KEESARA3                                              77.16   
LKDRM2                                                 0.00   
LKDRM4                                                 0.00   

Party Name  7 HILLS CONCRETE SOLUTIONS PVT LTD - GANDIMISAMMA  \
Warehouse                                                       
GIRMAPR2                                                 0.00   
GIRMAPUR                                              1329.26   
KEESARA3                                                 0.00   
LKDRM2                                                   0.00   
LKDRM4                                                   0.00   

Party Name  7HILLS INFRA CONCRETE - GANDIMAISAMMA  \
Warehouse                                           
GIRMAPR2                                     0.00   
GIRMAPUR                                   331.31   
KEESARA3                                     0.00   
LKDRM2                                       0.00   
LKDRM4                                       0.00   

Party Name  A & S BUILDR HOMETECH LLP - DAMMAIGUDA  A ANITHA - RAMPALLY  \
Warehouse                                                                 
GIRMAPR2                                       0.0                 0.00   
GIRMAPUR                                       0.0                 0.00   
KEESARA3                                       0.0               131.09   
LKDRM2                                         0.0                 0.00   
LKDRM4                                         0.0                 0.00   

Party Name  A H CONSTRUCTIONS - CHOWDARIGUDA  A KOMALA - BOWRAMPET  \
Warehouse                                                            
GIRMAPR2                                 0.0                503.34   
GIRMAPUR                                 0.0                  0.00   
KEESARA3                               253.4                  0.00   
LKDRM2                                   0.0                  0.00   
LKDRM4                                   0.0                  0.00   

Party Name  A KOMALA - BUCHUPALLY  A R ANMOL DEVELOPERS - PATANCHERUVU  \
Warehouse                                                                
GIRMAPR2                     0.00                                 0.00   
GIRMAPUR                  1265.38                                 0.00   
KEESARA3                     0.00                                 0.00   
LKDRM2                       0.00                              1768.02   
LKDRM4                       0.00                                 0.00   

Party Name  A R BULDERS - UPPAL  A R CONSTR

In [26]:
# Verify the cost after Optimization
total_after_opt = [decision_var_df.loc[w][p] * cost_mat.loc[w][p] * distance_matrix.loc[w][p] .sum().sum() for w in warehouses for p in party_names]

In [27]:
total1 = sum(total_after_opt)
print('total_cost_after_opt',total1)

total_cost_after_opt 277663817.8447723


In [28]:
# Before_Optimization the Transportation Cost in ₹
before_opt_cost = sand['Amount'].sum()
before_opt_cost

309961448.44037455

In [29]:
#Total Cost - Total Minimized Cost 
print("Difference_ before- after:",(before_opt_cost) -  sum(total_after_opt))

Difference_ before- after: 32297630.595602274


In [30]:
# Calculating the percentage 
print("percentage_decrease:",((((before_opt_cost) -  sum(total_after_opt)))/((before_opt_cost)))*100)

percentage_decrease: 10.41988633041737


In [31]:
# save the cost_matric in csv formate
cost_mat.to_csv('cost_mat_final_Customer_based.csv')

In [32]:
# save the distance_matric in csv formate
distance_matrix.to_csv('distance_final_customer_based.csv')

In [33]:
# save the decision_variable_matric in csv formate
decision_var_df.to_csv('final_decision_values_customer_based.csv')